In [1]:
import pandas as pd

df = pd.read_csv('public_result_label.csv')

# 질문 리스트
questions = df['question'].tolist()
answers = df['answer'].tolist()

df.head()

,question,answer,category02,label
0,여기 서명란에 펜이나 연필로 사인해도 되는 거 아니에요?,확인 결과 인감 외 사인도 동일 서명으로 처리됩니다.,민원 신청서 용어 질문,0
1,이런 전기 요금 할인처럼 비슷한 서비스 같은 것이 또 있나요?,네 소득기준과 세대원 특정 기준에 모두 충족하는 세대에게 에너지 바우처를 지원해 줍니다.,관내 복지 지원금 정보 질문,1
2,여기 소포 보낼 때 한 번에 결제하고 그런 서비스는 지원 안 하나요?,죄송합니다. 선불로 처리될 수밖에 없는 점 양해 부탁드립니다.,관내 복지 지원금 정보 질문,1
3,이런데 로봇이 있어? 신기하네 사업 지원금 관련 담당자는 어디 계실까요?,담당자분 성함 알려주시면 확인 후 안내 도와드리겠습니다.,입장 및 민원서비스 이용 안내,2
4,서류 가지고 왔어요. 여기에 제출하면 되죠?,성적장학금의 경우 재학 증명서와 성적 증명서가 필요합니다.,준비서류 확인,3


In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# fine-tuning된 모델과 토크나이저 로드
model = AutoModelForSequenceClassification.from_pretrained("./public_fine_tuned_model")
tokenizer = AutoTokenizer.from_pretrained("./public_fine_tuned_model")

c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
import torch.nn.functional as F
import numpy as np

# numpy의 _reconstruct 함수를 허용 목록에 추가
torch.serialization.add_safe_globals([np.core.multiarray._reconstruct])

# 신뢰할 수 있는 데이터 파일을 로드
question_embeddings = torch.load('embeddings/public_optimized_question_embeddings.pth')

# 평가 모드로 설정
model.eval()

def get_embedding(input_question, tokenizer, model):
    # 입력 문장을 토크나이즈
    inputs = tokenizer(input_question, return_tensors="pt", padding=True, truncation=True)
    
    with torch.no_grad():
        # hidden states를 포함하도록 설정
        outputs = model(**inputs, output_hidden_states=True)
        
        # 마지막 hidden state에서 [CLS] 토큰의 임베딩을 가져옴
        cls_embedding = outputs.hidden_states[-1][:, 0, :]  # [CLS] 토큰의 임베딩
        
    return cls_embedding.squeeze().numpy()  # numpy 배열로 반환



# 코사인 유사도를 계산하여 가장 유사한 답변을 찾는 함수
def find_most_similar_answer_cosine(input_question, question_embeddings, answers, tokenizer, model):
    # 입력 질문 임베딩 생성
    input_embedding = get_embedding(input_question, tokenizer, model)

    max_similarity = -1
    best_answer = None
    
    # 각 질문 임베딩과 유사도 비교
    for i, question_embedding in enumerate(question_embeddings):
        # question_embedding을 텐서로 변환하고 차원 맞추기
        question_embedding_tensor = torch.tensor(question_embedding).unsqueeze(0)  # (1, 768)
        
        # input_embedding도 텐서로 변환하고 차원 맞추기
        input_embedding_tensor = torch.tensor(input_embedding).unsqueeze(0)  # (1, 768)
        
        # 코사인 유사도 계산
        similarity = F.cosine_similarity(input_embedding_tensor, question_embedding_tensor).item()
        
        if similarity > max_similarity:
            max_similarity = similarity
            best_answer = answers[i]

    return best_answer, max_similarity  # 유사도 반환 추가

# 챗봇 응답 함수
def chatbot_response(input_question, tokenizer, model, question_embeddings, answers):
    # 1차 필터링: 분류 모델로 레이블 예측
    inputs = tokenizer(input_question, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=-1).item()

    # 2차 필터링: 같은 카테고리 내에서 코사인 유사도 계산
    # 같은 레이블의 질문들과 임베딩 필터링
    filtered_df = df[df['label'] == predicted_label]
    filtered_indices = filtered_df.index.tolist()

    # 필터링된 질문에 해당하는 미리 계산된 임베딩과 답변 가져오기
    filtered_question_embeddings = [question_embeddings[i] for i in filtered_indices]
    filtered_answers = [answers[i] for i in filtered_indices]

    # 코사인 유사도를 통해 가장 유사한 답변 찾기
    best_answer, cosine_similarity = find_most_similar_answer_cosine(input_question, filtered_question_embeddings, filtered_answers, tokenizer, model)
    
    return best_answer, cosine_similarity, predicted_label  # 세 가지 값 반환


c:\Temp\ipykernel_19400\650173660.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  question_embeddings = torch.load('embeddings/public_optimized_question_embeddings.pth')

In [4]:
# 예시 질문
input_question = "서류 제출은 어디에 하죠?"

# 챗봇 응답 호출
best_answer, cosine_similarity, predicted_label = chatbot_response(input_question, tokenizer, model, question_embeddings, answers)

# 결과 출력

print("예측된 레이블:", predicted_label)
print("최고 유사도 답변:", best_answer)
print("코사인 유사도:", cosine_similarity)

예측된 레이블: 4
최고 유사도 답변: 예약하신 경우 신청서와 구비서류를 접수처에 제출하시고 2층 영유아 모성 팀 앞에서 기다려 주시기 바랍니다.
코사인 유사도: 0.9992993474006653


In [6]:
# 예시 질문
input_question = "너무 목이 마른데요?"

# 챗봇 응답 호출
best_answer, cosine_similarity, predicted_label = chatbot_response(input_question, tokenizer, model, question_embeddings, answers)

# 결과 출력

print("예측된 레이블:", predicted_label)
print("최고 유사도 답변:", best_answer)
print("코사인 유사도:", cosine_similarity)



예측된 레이블: 2
최고 유사도 답변: 이용 가능 시간은 아홉 시부터 18시까지로 현재 민원접수 가능하십니다.
코사인 유사도: 0.9999954700469971
